In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import requests
from datetime import datetime

<IPython.core.display.Javascript object>

In [3]:
def _get_matched_recently(limit=1000, pure_number=-1, breed_count=-1, class_name=None):
    url = "https://api.crabada.com/public/order/matched-recently"
    params = {"limit": limit}
    result = requests.get(url=url, params=params)
    result.raise_for_status()

    json = result.json()

    crabadas = json["result"]

    def check_crabada(cra):
        if pure_number != -1 and cra["pure_number"] != pure_number:
            return False
        if breed_count != -1 and cra["breed_count"] != breed_count:
            return False
        if class_name is not None and cra["class_name"] != class_name:
            return False
        return True

    filtered_crabadas = [cra for cra in crabadas if check_crabada(cra)]
    return filtered_crabadas

<IPython.core.display.Javascript object>

In [7]:
crabadas = _get_matched_recently(
    limit=1000, pure_number=6, breed_count=2, class_name=None
)

<IPython.core.display.Javascript object>

In [8]:
import pandas as pd
import plotly.express as px


x = [datetime.fromtimestamp(cra["transaction_time"]) for cra in crabadas]
y = [cra["price"] / 10**18 for cra in crabadas]
color = [cra["class_name"] for cra in crabadas]
hover_data = [
    f"{cra['class_name']}-P{cra['pure_number']}-B{cra['breed_count']}"
    for cra in crabadas
]

fig = px.scatter(x=x, y=y, color=color, hover_name=hover_data)
fig.show()

<IPython.core.display.Javascript object>

In [9]:
group_by_class = {}
for cra in crabadas:
    class_name = cra["class_name"]
    if class_name in group_by_class:
        group_by_class[class_name] = group_by_class[class_name] + 1
    else:
        group_by_class[class_name] = 1

fig = px.bar(x=group_by_class.keys(), y=group_by_class.values())
fig.show()

<IPython.core.display.Javascript object>